In [ ]:
import torch
from torch import Tensor

In [ ]:
def trilateration_loss(guess: Tensor, ref_points: Tensor, distances: Tensor) -> Tensor:
    """
    Compute the loss function for 2D trilateration. The loss is the squared difference
    between the actual distances and the guessed distances to the reference points.

    :param guess: Tensor of shape (2,) representing the guessed target location (x, y).
    :param ref_points: Tensor of shape (N, 2) representing the coordinates of the reference points.
    :param distances: Tensor of shape (N,) representing the distances to the target from the
    reference points.
    :return: The loss value, a scalar tensor.
    """
    guessed_distances = torch.sqrt(torch.sum((ref_points - guess) ** 2, dim=1))
    loss = torch.sum((guessed_distances - distances) ** 2)
    return loss

In [ ]:
def trilateration(
    ref_points: Tensor,
    distances: Tensor,
    lr: float = 1e-2,
    num_iters: int = 1000
) -> Tensor:
    """
    Perform 2D trilateration using gradient descent to find the target location.

    :param ref_points: Tensor of shape (N, 2) representing the coordinates of the reference points.
    :param distances: Tensor of shape (N,) representing the distances to the target from the
    reference points.
    :param lr: Learning rate for gradient descent.
    :param num_iters: Number of iterations to run the gradient descent.
    :return: A tensor of shape (2,) representing the optimized target location (x, y).
    """
    guess = torch.randn(2, requires_grad=True)
    optimizer = torch.optim.SGD([guess], lr=lr)

    for _ in range(num_iters):
        optimizer.zero_grad()
        loss = trilateration_loss(guess, ref_points, distances)
        loss.backward()
        optimizer.step()

    return guess.detach()

In [ ]:
reference_points = torch.tensor([[2, -2], [10, 8], [-1, 6]], dtype=torch.float32)
distances_to_target = torch.tensor([4, 10, 5], dtype=torch.float32)

target_location = trilateration(reference_points, distances_to_target)

print(f"Estimated target location: {target_location}")